In [1]:
# Read in Libraries
from __future__ import division, print_function
from logbook import Logger, StreamHandler
import sys
StreamHandler(sys.stdout).push_application()
log = Logger('Logbook')
import shutil, csv, time
timestr = time.strftime("%Y%m%d")
import ujson as json
import utils; reload(utils)
from utils import *
import gc
# from __future__ import division, print_function
from theano.sandbox import cuda
from vgg16bn import Vgg16BN
from sklearn import metrics
%matplotlib inline

def accuracyfunc(y_act, y_pred):
    return metrics.accuracy_score(np.argmax(y_act, axis=1), np.argmax(y_pred, axis=1))
    
def refresh_directory_structure(name, sub_dirs):
    gdir = os.path.join(path, name)
    if os.path.exists(gdir):
        shutil.rmtree(gdir)
    os.makedirs(gdir)
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(gdir, sub_dir))

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
# Set Parameters and check files
refresh_directories = False
input_exists = False
full = False
log.info('Set Paramters')
path = "../data/fish/crop/"
batch_size=32
clip = 0.99
bags = 1
load_size = (300,300)#(360, 640)

[2017-02-26 23:01:14.266206] INFO: Logbook: Set Paramters


In [3]:
# Read in our VGG pretrained model
log.info('Get VGG')
model = vgg_ft_bn(8)

# Create our VGG model
log.info('Create VGG')
vgg640 = Vgg16BN(load_size).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# get labels
(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

# Read in filenames
log.info('Read filenames')
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

[2017-02-26 23:01:14.278816] INFO: Logbook: Get VGG
[2017-02-26 23:01:17.533696] INFO: Logbook: Create VGG
Found 1962 images belonging to 8 classes.
Found 467 images belonging to 8 classes.
Found 721 images belonging to 1 classes.
[2017-02-26 23:01:18.068704] INFO: Logbook: Read filenames


In [4]:
log.info('Read in data')
if not input_exists:

    batches = get_batches(path+'train', batch_size=batch_size)
    val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
    (val_classes, trn_classes, val_labels, trn_labels, 
        val_filenames, filenames, test_filenames) = get_classes(path)
    
    # Fetch our large images 
    # Precompute the output of the convolutional part of VGG
    log.info('Fetch images')
    log.info('Get VGG output')
    log.info('Write VGG output')
    
    test = get_data(path+'test', load_size)
    conv_test_feat = vgg640.predict(test, batch_size=16, verbose=1)
    save_array(path+'../results/conv_test_crop_feat.dat', conv_test_feat)     
    del test, conv_test_feat
    gc.collect()
    
    val = get_data(path+'valid', load_size)
    conv_val_feat = vgg640.predict(val, batch_size=16, verbose=1)
    save_array(path+'../results/conv_val_crop_feat.dat', conv_val_feat)
    del val, conv_val_feat
    gc.collect()
    
    trn = get_data(path+'train', load_size)
    conv_trn_feat = vgg640.predict(trn, batch_size=16, verbose=1)    
    del trn
    gc.collect()
    save_array(path+'../results/conv_trn_crop_feat.dat', conv_trn_feat) 
    del conv_trn_feat
    gc.collect()

    # For memory purposes delete out the original train and validation
    log.info('Clear up memory')
    #del trn, val, test
    gc.collect()
    gc.collect()

[2017-02-26 23:01:18.098764] INFO: Logbook: Read in data
Found 1962 images belonging to 8 classes.
Found 467 images belonging to 8 classes.
Found 1962 images belonging to 8 classes.
Found 467 images belonging to 8 classes.
Found 721 images belonging to 1 classes.
[2017-02-26 23:01:18.128145] INFO: Logbook: Fetch images
[2017-02-26 23:01:18.128868] INFO: Logbook: Get VGG output
[2017-02-26 23:01:18.129516] INFO: Logbook: Write VGG output
Found 721 images belonging to 1 classes.
721/721 [==============================] - 31s    
Found 467 images belonging to 8 classes.
467/467 [==============================] - 20s    
Found 1962 images belonging to 8 classes.
1962/1962 [==============================] - 85s    
[2017-02-26 23:04:33.354623] INFO: Logbook: Clear up memory


In [5]:
gc.collect()
conv_val_feat = load_array(path+'../results/conv_val_crop_feat.dat')
conv_trn_feat = load_array(path+'../results/conv_trn_crop_feat.dat') 
conv_test_feat = load_array(path+'../results/conv_test_crop_feat.dat')

In [6]:
if full:
    conv_trn_feat = np.concatenate([conv_trn_feat, conv_val_feat])
    trn_labels = np.concatenate([trn_labels, val_labels]) 
    #trn_bbox = np.concatenate([trn_bbox, val_bbox])
    
# Our Convolutional Net Architecture
log.info('Create and fit CNN')
p=0.6
# Set up the fully convolutional net (FCN); 
conv_layers,_ = split_at(vgg640, Convolution2D)
nf=128; p=0. # No dropout

[2017-02-26 23:04:35.410978] INFO: Logbook: Create and fit CNN


In [22]:
conv_layers[-1].output_shape[1:]

(512, 18, 18)

In [ ]:
gc.collect()
nf = 1024
p  = 0.4
def create_model():
    inp = Input(conv_layers[-1].output_shape[1:])
    x = MaxPooling2D()(inp)
    x = ZeroPadding2D((1,1))(x)
    x = Convolution2D(256,3,3, activation='relu', border_mode='same')(x)
    x =   Dropout(p)(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = ZeroPadding2D((1,1))(x)
    x = Convolution2D(256,3,3, activation='relu', border_mode='same')(x)
    x =   Dropout(p)(x)
    x = BatchNormalization(axis=1)(x)
    x = MaxPooling2D()(x)
    x = ZeroPadding2D((1,1))(x)
    x = Convolution2D(256,3,3, activation='relu', border_mode='same')(x)
    x =   Dropout(p)(x)
    x = BatchNormalization(axis=1)(x)
    x1 =   MaxPooling2D()(x)
    x1 =   Convolution2D(8,3,3, border_mode='same')(x1)
    #x1 =   Dropout(p/2)(x1)
    x1 =   GlobalAveragePooling2D()(x1)
    x_class = Dense(8, activation='softmax', name='class')(x1)
    return inp, x_class


model = []
predsls = []
pvalsls = []

for ii in range(5):
    inp, x_class = create_model()
    model.append(Model([inp], [x_class]))
    model[ii].compile(Adam(lr=1e-4), loss=['categorical_crossentropy'], metrics=['accuracy'])
    model[ii].fit(conv_trn_feat, [trn_labels], batch_size=batch_size, nb_epoch=40, 
                 validation_data=(conv_val_feat, [val_labels]))
    
    model[ii].optimizer.lr = 1e-5
    model[ii].fit(conv_trn_feat, [trn_labels], batch_size=batch_size, nb_epoch=40, 
                 validation_data=(conv_val_feat, [val_labels]))

    model[ii].optimizer.lr = 1e-7
    model[ii].fit(conv_trn_feat, [trn_labels], batch_size=batch_size, nb_epoch=1, 
                 validation_data=(conv_val_feat, [val_labels]))

    count = 0
    while count < 5:
        model[ii].fit(conv_trn_feat, [trn_labels], batch_size=batch_size, nb_epoch=1, 
                     validation_data=(conv_val_feat, [val_labels]))
        predsls.append(model[ii].predict(conv_test_feat, batch_size=batch_size)) # or try 32 batch_size
        pvalsls.append(model[ii].predict(conv_val_feat, batch_size=batch_size))
        val_score = "%.3f" % metrics.log_loss(val_labels, sum(pvalsls)/len(pvalsls))
        acc_score = "%.3f" % accuracyfunc(val_labels, do_clip(sum(pvalsls)/len(pvalsls), clip))
        log.info('Bagged Validation Logloss ' + str(val_score))
        log.info('Bagged Validation Accuracy ' + str(acc_score))
        count += 1

th
Train on 1962 samples, validate on 467 samples
Epoch 1/40
1962/1962 [==============================] - 3s - loss: 1.4658 - acc: 0.4888 - val_loss: 1.9579 - val_acc: 0.3383
Epoch 2/40
1962/1962 [==============================] - 3s - loss: 0.8884 - acc: 0.6972 - val_loss: 1.5610 - val_acc: 0.4818
Epoch 3/40
1962/1962 [==============================] - 3s - loss: 0.5728 - acc: 0.8165 - val_loss: 1.4573 - val_acc: 0.5268
Epoch 4/40
1962/1962 [==============================] - 3s - loss: 0.3734 - acc: 0.8930 - val_loss: 1.3357 - val_acc: 0.5589
Epoch 5/40
1962/1962 [==============================] - 3s - loss: 0.2762 - acc: 0.9241 - val_loss: 1.2803 - val_acc: 0.5782
Epoch 6/40
1962/1962 [==============================] - 3s - loss: 0.1890 - acc: 0.9501 - val_loss: 1.1962 - val_acc: 0.6253
Epoch 7/40
1962/1962 [==============================] - 3s - loss: 0.1326 - acc: 0.9669 - val_loss: 1.2109 - val_acc: 0.6103
Epoch 8/40
1962/1962 [==============================] - 3s - loss: 0.1068 -

In [8]:
# metrics.log_loss(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .9999))
preds = sum(predsls)/len(predsls)
subm = do_clip(preds, clip)

if full:
    subm_name = path+'../results/subm_full_crop_conv_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = path+'../results/subm_part_crop_conv_' + timestr + '.csv' #'.csv.gz'

classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.to_csv(subm_name, index=False)#, compression='gzip')
log.info('Done - files @ ' + subm_name)

ZeroDivisionError: division by zero

In [ ]:
FileLink(subm_name)